In [1]:
import numpy as np
import pandas as pd
%matplotlib inline 
import re
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
plt.rcParams['figure.figsize'] = (8, 6)
import xlsxwriter
import scipy.stats as stats

In [84]:
#name of experiment
name = 'Multirun_SearchLikesInjection_run1'
# files with payments of all users
tg = pd.read_csv(r'C:\Users\user\Downloads\report-1524044904.csv')
cg1 = pd.read_csv(r'C:\Users\user\Downloads\report-1524041913.csv')
cg2 = pd.read_csv(r'C:\Users\user\Downloads\report-1524041930.csv')
tg['group'] = 'tg'
cg1['group'] = 'cg1'
cg2['group'] = 'cg2'
df= tg.append(cg1.append(cg2))

In [85]:
df['like_sent-fake_0_1day'] = df['like_sent_0_1day'] - df['fake_like_sent_0_1day']
df['like_sent-fake_1day_2day'] = df['like_sent_1day_2day'] - df['fake_like_sent_1day_2day']
df['like_sent-fake_2day_3day'] = df['like_sent_2day_3day'] - df['fake_like_sent_2day_3day']
df['like_sent-fake_3day_4day'] = df['like_sent_3day_4day'] - df['fake_like_sent_3day_4day']
df['mutual-fake_0_1day'] = df['mutual_0_1day'] - df['fake_mutual_0_1day']
df['mutual-fake_1day_2day'] = df['mutual_1day_2day'] - df['fake_mutual_1day_2day']
df['mutual-fake_2day_3day'] = df['mutual_2day_3day'] - df['fake_mutual_2day_3day']
df['mutual-fake_3day_4day'] = df['mutual_3day_4day'] - df['fake_mutual_3day_4day']

In [86]:
df = df[df['sex'] == 1]
df.transpose().head(100)

,1,2,3,4,5,6,8,9,10,11,...,19501,19502,19503,19505,19506,19507,19508,19509,19510,19512
user_id,117821181,117803455,117844777,117801709,117859433,117841021,117848259,117831263,117802868,117869360,...,117856788,117846126,117850656,117843342,117828580,117827004,117843390,117844331,117856400,117866112
sex,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
isReferral,1,0,0,1,1,1,0,0,1,1,...,0,0,1,1,1,1,1,0,0,1
isPaid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
active_days_0_1day,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
active_days_1day_2day,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,1,1,0,0,0
active_days_2day_3day,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
active_days_3day_4day,0,0,0,0,0,1,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0
like_sent_0_1day,4,8,0,3,6,0,0,433,2,33,...,113,1,153,1,0,7,14,35,24,6
like_sent_1day_2day,0,0,0,0,0,0,0,9,0,0,...,0,0,0,0,29,33,54,0,0,0


In [5]:
def mean_and_part(value, period):
        a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
        b =(pd.DataFrame(df[(df['active_days' + period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        return a.append(b)

In [80]:
def mean_and_part2(value, period):
        a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
        b =(pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        c = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value+ '-fake' + period].mean()).transpose()
        d = pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value+ '-fake' + period] >0)].groupby('group')['user_id'].count().transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        return a.append(b.append(c.append(d)))

In [81]:
def summary_statistics(period):
    b = mean_and_part2('mutual', period)
    c = mean_and_part2('like_sent', period)
    d = mean_and_part('like_received', period)
    e = mean_and_part('message_sent', period)
    f = mean_and_part('message_received', period)
    g = mean_and_part('first_dialog_started', period)
    h = mean_and_part('fist_dialog_received', period)
    k = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose()
    
    m = pd.concat([b,c,d,e,f,g,h,k],axis = 0, join = 'inner')
    m['dif'] = (m['tg']/((m['cg1']+m['cg2'])/2)-1)
    cols = ['tg', 'cg1', 'cg2', 'dif']
    m = m[cols]
    m.columns = ['TG', 'CG1', 'CG2', 'dif']
    m.index = ['mutual', 'mutual_nonzero_part', 'mutual-fake', 'mutual-fake_nonzero_part',
                'like_sent', 'like_sent_nonzero_part', 'like_sent-fake', 'like_sent-fake_nonzero_part','like_received',
                'like_received_nonzero_part', 'message_sent', 'message_sent_nonzero_part',
                'message_received', 'message_receive_nonzero_part',
                'first_dialog_started', 'first_dialog_started_nonzero_part',
                'fist_dialog_received', 'fist_dialog_received_nonzero_part', 'active_users']
    cm = sns.light_palette("yellow", as_cmap=True)
    return (m.fillna(0))

In [87]:
m1 = summary_statistics('_0_1day')
m2 = summary_statistics('_1day_2day')
m3 = summary_statistics('_2day_3day')
m4 = summary_statistics('_3day_4day')
m = pd.concat([m1,m2,m3,m4], axis = 1)
m.columns = ['TG_1', 'CG1_1', 'CG2_1', 'dif_1',
             'TG_2', 'CG1_2', 'CG2_2', 'dif_2',
             'TG_3', 'CG1_3', 'CG2_3', 'dif_3',
             'TG_4', 'CG1_4', 'CG2_4', 'dif_4']
cm = sns.light_palette("yellow", as_cmap=True)
m.fillna(0).style.format({'dif_1': '{:.2%}', 'dif_2': '{:.2%}',
                'dif_3': '{:.2%}', 'dif_4': '{:.2%}'}).background_gradient(subset=['dif_1', 'dif_2', 'dif_3', 'dif_4'], cmap=cm)

,TG_1,CG1_1,CG2_1,dif_1,TG_2,CG1_2,CG2_2,dif_2,TG_3,CG1_3,CG2_3,dif_3,TG_4,CG1_4,CG2_4,dif_4
mutual,1.33088,0.985977,1.02389,32.43%,3.15111,2.48591,2.59509,24.03%,2.44798,2.01364,1.84185,26.99%,2.30003,1.72406,1.77398,31.50%
mutual_nonzero_part,0.222822,0.202844,0.201943,10.09%,0.420019,0.370119,0.383732,11.43%,0.365271,0.323012,0.317902,13.98%,0.341448,0.296308,0.307317,13.13%
mutual-fake,1.02303,0.884816,0.929382,12.78%,2.28027,2.10414,2.22284,5.40%,1.61932,1.54526,1.52341,5.54%,1.3582,1.20401,1.48976,0.84%
mutual-fake_nonzero_part,0.195498,0.190647,0.190457,2.60%,0.356689,0.339227,0.3491,3.64%,0.297971,0.28995,0.285791,3.51%,0.27632,0.263526,0.275772,2.47%
like_sent,49.4699,46.4588,46.0399,6.96%,52.03,48.9866,50.5185,4.58%,30.8808,30.8859,29.2467,2.71%,26.6453,24.3997,27.6371,2.41%
like_sent_nonzero_part,0.703207,0.701865,0.694034,0.75%,0.572281,0.571789,0.555907,1.50%,0.480092,0.47048,0.478994,1.13%,0.444831,0.436665,0.437073,1.82%
like_sent-fake,49.0845,45.9574,45.8025,6.98%,51.4914,48.3175,50.2321,4.50%,30.6848,30.5555,28.9971,3.05%,26.4477,23.9952,27.3291,3.06%
like_sent-fake_nonzero_part,0.699824,0.697691,0.691147,0.78%,0.563234,0.564795,0.549575,1.09%,0.472386,0.462608,0.470698,1.23%,0.438192,0.430617,0.431545,1.65%
like_received,0.472578,0.502935,0.464724,-2.33%,1.06583,1.13212,1.12013,-5.35%,0.73645,0.936237,0.819909,-16.13%,0.565918,0.715468,0.680976,-18.95%
like_received_nonzero_part,0.138312,0.137686,0.134803,1.52%,0.283734,0.266369,0.283198,3.26%,0.245826,0.242981,0.239229,1.96%,0.202656,0.200509,0.209431,-1.13%


In [40]:
mean_and_part2('like_sent', '_0_1day')

group,cg1,cg2,tg
like_sent_0_1day,44.980393,44.378155,46.602474
user_id,0.674052,0.663814,0.674272
0,38.084796,37.466011,39.983342
user_id,0.524896,0.513557,0.526993
